In [3]:
!/gws/pw/j05/cop26_hackathons/bristol/install-kernel

Installed kernelspec cmip6 in /home/users/fdmv/.local/share/jupyter/kernels/cmip6
Installed cmip6 Jupyter kernel for the hackathon


In [2]:
def compute_time_average_fluxes_master(expc_data_loc,fluxes_data_name,timeslice_times):
# VERSION 1: def compute_time_average_fluxes_master(expc_data_loc,flux_exp_data_name,flux_transf_data_name,timeslice_times):
    
    # INPUTS - Version 1
	# expc_data_loc - full path to netcdf file
	# flux_exp_data_name - netcdf fluxt at export depth file name
	# flux_transf_data_name - netcdf flux at transfer depth file name
	# timeslices_times - start and finish times of timesclice averaging in pairs, e.g., ["1960","1970"] or ["1960","1970","1980","1990"]

    # INPUTS - Version 2
	# expc_data_loc - full path to netcdf file
	# fluxes_data_name - netcdf flux at all depths file name
	# timeslices_times - start and finish times of timesclice averaging in pairs, e.g., ["1960","1970"] or ["1960","1970","1980","1990"]
    
    # OUTPUTS - Version 1
	# flux_exp_timeseries_flux_exp_data_name.nc
    # flux_transf_timeseries_flux_transf_data_name.nc
	# flux_exp_timeslice_start_timeslice_finish_flux_exp_data_name.nc
    # flux_transf_timeslice_start_timeslice_finish_flux_transf_data_name.nc
    
    # OUTPUTS - Version 2
	# fluxes_timeseries_flux_exp_data_name.nc
	# fluxes_timeslice_start_timeslice_finish_flux_exp_data_name.nc

        
        print('>>> loading flux and area data')
        ### load data
#        flux_exp=xr.open_mfdataset(expc_data_loc+flux_exp_data_name)
#        flux_transf=xr.open_mfdataset(expc_data_loc+flux_transf_data_name)
        fluxes = xr.open_mfdataset(expc_data_loc+fluxes_data_name)
        area = xr.open_mfdataset('/badc/cmip6/data/CMIP6/CMIP/MOHC/UKESM1-0-LL/piControl/r1i1p1f2/Ofx/areacello/gn/latest/areacello_Ofx_UKESM1-0-LL_piControl_r1i1p1f2_gn.nc')

        print('>>> calculating global annual averages for fluxes (sloooow)')
        ### annual averaging
        # compute here to save memory in following steps
#        flux_exp_data=flux_exp.rolling(time=12, center=True).mean().compute()
#        flux_transf_data=flux_transf.rolling(time=12, center=True).mean().compute()
        
        fluxes_data=fluxes['expc']
        #fluxes_data_mean=fluxes_data.rolling(time=12, center=True).mean().compute()
        fluxes_data_mean=fluxes_data.rolling(time=12, center=True).mean().compute()
        
        level = fluxes['lev']
        level_data = level.compute()
    
        # computing fractional area correction
        area_data = area['areacello']
        area_data.compute()
        weights = (area_data/(area_data.sum(dim=['i','j']))).compute()
        
        # convert to global mean timeseries
#        weighted_flux_exp_data = flux_exp_data.weighted(weights)
#        weighted_flux_transf_data = flux_transf_data.weighted(weights)

        weighted_fluxes_data_mean = (fluxes_data_mean*weights).compute()
        
        print('>>> output flux data - 1')
        # convert to global mean timeseries
#        flux_exp_timeseries=weighted_flux_exp_data.mean(dim=['i','j'])
#        flux_exp_timeseries.to_netcdf('timeseries_'+flux_exp_data_name)
#        flux_transf_timeseries=weighted_flux_transf_data.mean(dim=['i','j'])
#        flux_transf_timeseries.to_netcdf('timeseries_'+flux_transf_data_name)

        fluxes_timeseries=weighted_fluxes_data_mean.mean(dim=['i','j'])
        fluxes_timeseries.to_netcdf('timeseries_'+fluxes_data_name)
        
        print('>>> output flux data - 2')
        # get flux efficiency at each layer
        layer_ratio_fluxes_timeseries = fluxes_timeseries*0
        for depth in range(1,len(level_data)-1):
            layer_ratio_fluxes_timeseries[:,depth-1]=fluxes_timeseries[:,depth]/fluxes_timeseries[:,depth-1]
            
        layer_ratio_fluxes_timeseries.to_netcdf('layer_ratio_timeseries_'+fluxes_data_name)
        
        print('>>> output flux data - 3')
        # get timeslice data
        for n in range(0,len(timeslice_times),2):
            fluxes_timeslice=fluxes_data_mean.sel(time=slice(timeslice_times[n], timeslice_times[n+1])).mean(dim='time')
            fluxes_timeslice.to_netcdf('timeslice_'+timeslice_times[n]+'_'+timeslice_times[n+1]+fluxes_data_name)
#            flux_exp_timeslice=flux_exp.sel(time=slice(timeslice_times[n], timeslice_times[n+1])).mean(dim='time')
#            flux_exp_timeslice.to_netcdf('timeslice_'+timeslice_times[n]+'_'+timeslice_times[n+1]+flux_exp_data_name)
#            flux_transf_timeslice=flux_transf.sel(time=slice(timeslice_times[n], timeslice_times[n+1])).mean(dim='time')
#            flux_transf_timeslice.to_netcdf('timeslice_'+timeslice_times[n]+'_'+timeslice_times[n+1]+flux_transf_data_name)
        print('>>> Done! :)')
###### END #####  

In [ ]:
# Loading modules, etc
from itertools import chain
from glob import glob

import matplotlib.pyplot as plt
import xarray as xr

#compute_time_average_fluxes_master('/home/users/fdmv/OthersCode/','flux_depth_exp_interp_expc_Omon_UKESM1-0-LL_piControl_r1i1p1f2_gn_196001-199912.nc','flux_depth_transf_interp_expc_Omon_UKESM1-0-LL_piControl_r1i1p1f2_gn_196001-199912.nc',["1960","1970"])
compute_time_average_fluxes_master('/gws/pw/j05/cop26_hackathons/bristol/project03/input_nc/UKESM1-0-LL/','expc_Omon_UKESM1-0-LL_piControl_r1i1p1f2_gn_196001-199912.nc',["1960","1970"])

>>> loading flux and area data
>>> calculating global annual averages for fluxes (sloooow)
